# <center> Bit Level Manipulation
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Instructor: Dr. Jeremy Daily

## Learning Objectives
By the end of this lesson, students should be able to:
1. Understand bitwise boolean operations
2. Be able to apply masks and shifts to determine switch states.
3. Write code to set bits, clear bits, and toggle bits.

## Switch States
Many vehicle and industrial communications are for switches, which take 2 bits (four states). These states are often:

00 = off

01 = on

10 = error

11 = not used

In [1]:
# Given a J1939 Message
msg = '(012.565071)  can1  18FEF100   [8]  FF 00 00 FC FF 68 00 CF'
msg_bytes = bytes.fromhex(msg[-24:])
msg_bytes

b'\xff\x00\x00\xfc\xffh\x00\xcf'

In [2]:
# Bytes are immutable, meaning you can't change them
msg_bytes[1] = 35

TypeError: 'bytes' object does not support item assignment

In [3]:
# Make a mutable bytearray instead
msg_byte_array = bytearray.fromhex(msg[-24:])
msg_byte_array

bytearray(b'\xff\x00\x00\xfc\xffh\x00\xcf')

In [4]:
# Byte arrays are mutable
msg_byte_array[1] = 0x8D
msg_byte_array

bytearray(b'\xff\x8d\x00\xfc\xffh\x00\xcf')

We don't want to change the message, so we'll stick with the bytes data type.

In [5]:
# Compute the PGN
split_message =  msg.split()
split_message

['(012.565071)',
 'can1',
 '18FEF100',
 '[8]',
 'FF',
 '00',
 '00',
 'FC',
 'FF',
 '68',
 '00',
 'CF']

### J1939 CAN Messages
Please download the SAE J1939-21 document from the CSU Libary:

https://saemobilus-sae-org.ezproxy2.library.colostate.edu/content/J1939/21_202205



In a J1939 message, there are 29 bits in the ID. They can be broken into 4 parts: Priority, PDU Format, PDU Specific, and Source Address. To extract these from the ID, we'll cover 2 approaches:
1. Convert to raw bytes and use the struct.unpack method to determine values.
2. Convert to an integer and use masks and shifts to determine the values.

In [6]:
import struct

In [7]:
can_id = bytes.fromhex(split_message[2])
can_id

b'\x18\xfe\xf1\x00'

In [8]:
#PDU = Protocol Data Unit
(PR,PF,PS,SA) = struct.unpack("BBBB",can_id)
print("Source Address = {}".format(SA))
print("PDU Specific   = {}".format(PS))
print("PDU Format     = {}".format(PF))
print("Priority       = {}".format(PR))

Source Address = 0
PDU Specific   = 241
PDU Format     = 254
Priority       = 24


In [9]:
# Show the bits
print("{:08b}".format(PR))

00011000


In [10]:
#However, priority is only three bits long (0-7) in J1939, 
# so we have to look at masking off the top 3 bits and shifting them
priority = (PR & 0b00011100) >> 2
priority

6

This approach requires additional granualrity since we just dropped the 2 bits in PR byte. Let's try an approach of masking and bit shifting of the integer.

In [11]:
can_id = int(split_message[2],16)
can_id

419361024

In [12]:
print("{:08X}".format(can_id))

18FEF100


In [13]:
# Source address 
SA = (can_id & 0x000000000FF)
SA

0

In [14]:
#Mask for the 2nd byte
PS = (can_id & 0xFF00)
PS

61696

In [15]:
# Show as hex characters
print("{:X}".format(PS))

F100


In [16]:
# We actually need to shift the result by one byte, or 8 bits
PS = (can_id & 0xFF00) >> 8
PS

241

In [17]:
# Show as hex characters
print("{:X}".format(PS))

F1


In [18]:
# Now lets include the extra 2 bits in the PF determination
mask = 0x3FF0000
print("Mask: {:032b}".format(mask))
print("Data: {:032b}".format(can_id))

Mask: 00000011111111110000000000000000
Data: 00011000111111101111000100000000


Perform a bit-wise AND operation. 

Everywhere there is a zero in the mask the result is zeroed.

Everywhere there is a 1 in the mask, the result is the data.


In [19]:
PF = mask & can_id
print("Mask: {:032b}".format(mask))
print("Data: {:032b}".format(can_id))
print("Rslt: {:032b}".format(PF))
PF

Mask: 00000011111111110000000000000000
Data: 00011000111111101111000100000000
Rslt: 00000000111111100000000000000000


16646144

In [20]:
# Now shift the AND result the appropriate number of bits (16)
PF = (can_id & mask) >> 16
PF

254

In [21]:
# Show as hex characters
print("{:X}".format(PF))

FE


In [22]:
priority_mask = 0x1C000000
PR = priority_mask & can_id
print("Mask: {:032b}".format(priority_mask))
print("Data: {:032b}".format(can_id))
print("Rslt: {:032b}".format(PR))

Mask: 00011100000000000000000000000000
Data: 00011000111111101111000100000000
Rslt: 00011000000000000000000000000000


In [23]:
# Now shift the bits over 26 times (8 + 8 + 8 + 2)
priority = PR >> 26
priority

6

The goal is to get the parameter group number (PGN) from the ID. In J1939, when the PF is 240 or above, the PGN is the PF and PS concatenated.

In [24]:
PGN = PF << 8 + PS
PGN

229774927080299325293648673376615067146332625820880494265798643265702616629248

The order of operations is challenging. 
https://en.cppreference.com/w/c/language/operator_precedence
Addition takes place before bit shifting. But bit shifting takes place before an and operation. 

In [25]:
# Order of operations matter:
PGN =  (PF << 8) + PS
PGN

65265

The J1939 Standard has this to say about PGN 65265 (0xFEF1):

Name = Cruise Control/ Vehicle Speed 1

Acronym = CCVS1

It has the following suspect parameter numbers (SPNs)

| Position | SPN	| SP Label |SP Length | Resolution
| --- | --- |:------- | --- | --- |
| 1.1 | 69	 | Two Speed Axle Switch 	                 | 2 bits	| 4 states/2 bit
| 1.3 | 70	 | Parking Brake Switch	                     | 2 bits	| 4 states/2 bit
| 1.5 | 1633 | Cruise Control Pause Switch	             | 2 bits	| 4 states/2 bit
| 1.7 | 3807 | Park Brake Release Inhibit Request	     | 2 bits	| 4 states/2 bit
| 2-3 | 84	 | Wheel-Based Vehicle Speed	             | 2 bytes| 1/256 km/h per bit
| 4.1 | 595	 | Cruise Control Active	                 | 2 bits	| 4 states/2 bit
| 4.3 | 596	 | Cruise Control Enable Switch	             | 2 bits	| 4 states/2 bit
| 4.5 | 597	 | Brake Switch	                             | 2 bits	| 4 states/2 bit
| 4.7 | 598	 | Clutch Switch	                         | 2 bits	| 4 states/2 bit
| 5.1 | 599	 | Cruise Control Set Switch	             | 2 bits	| 4 states/2 bit
| 5.3 | 600	 | Cruise Control Coast (Decelerate) Switch	 | 2 bits	| 4 states/2 bit
| 5.5 | 601	 | Cruise Control Resume Switch	             | 2 bits	| 4 states/2 bit
| 5.7 | 602	 | Cruise Control Accelerate Switch	         | 2 bits	| 4 states/2 bit
| 6	  | 86 	 | Cruise Control Set Speed	                 | 1 byte	| 1 km/h per bit
| 7.1 | 976	 | PTO Governor State	                     | 5 bits	| 32 states/5 bit
| 7.6 | 527	 | Cruise Control States	                 | 3 bits	| 8 states/3 bit
| 8.1 | 968	 | Engine Idle Increment Switch	             | 2 bits	| 4 states/2 bit
| 8.3 | 967	 | Engine Idle Decrement Switch	             | 2 bits	| 4 states/2 bit
| 8.5 | 966	 | Engine Diagnostic Test Mode Switch	     | 2 bits	| 4 states/2 bit
| 8.7 | 1237 | Engine Shutdown Override Switch	         | 2 bits	| 4 states/2 bit


In [26]:
#Let's decode the message.
# Recall
msg_bytes

b'\xff\x00\x00\xfc\xffh\x00\xcf'

In [27]:
# Let's find our brake switch, 4.5
# In J1939 the index is from 1, in Python and C, it's from 0.
# We are looking for byte 3
byte_of_interest = msg_bytes[3]
print("{:08b}".format(byte_of_interest))

11111100


In [28]:
# Position 5 is actually 4 in python. 
mask = 0b00110000
print("{:08b}".format(mask))

00110000


In [29]:
#Apply the mask:
result = byte_of_interest & mask
print("{:08b}".format(mask))
print("{:08b}".format(byte_of_interest))
print("{:08b}".format(result))

00110000
11111100
00110000


In [30]:
SPN597_value = result >> 4
SPN597_value

3

In [31]:
# Per J1939
SPN597_meanings = {0:"Brake pedal released",
                   1:"Brake pedal depressed",
                   2:"Error",
                   3:"Not Available"}
print(SPN597_meanings[SPN597_value])

Not Available


The brake switch is monitored by the instrument panel, not the engine.

## Bit Manipulation
Often we have to set the values of certain bits based of inputs or state changes. There may be many switch states in 1 byte, so only the bit of interest is able to change.
https://stackoverflow.com/questions/47981/how-do-you-set-clear-and-toggle-a-single-bit

In [32]:
# Let's start with 4 switches represented by a single byte.
switch = 0
sw1 = 0
sw2 = 1
sw3 = 0
sw4 = 1
# Initial build
switch = sw1 + (sw2 << 2) + (sw3 << 4) + (sw4 << 6)
print("{:08b}".format(switch))

01000100


In [33]:
struct.pack("B",switch)

b'D'

In [34]:
# Let's set Switch 1 (OR)
switch |= 1
print("{:08b}".format(switch))

01000101


In [35]:
# Let's set Switch 3
switch |= 1 << 4
print("{:08b}".format(switch))

01010101


In [36]:
# Clear Switch 4 (Inverted AND)
switch &= ~(1 << 6)
print("{:08b}".format(switch))

00010101


In [37]:
# Clear Switch 2 (Inverted AND)
switch &= ~(1 << 2)
print("{:08b}".format(switch))

00010001


In [38]:
# Toggle Switch 3 (XOR)
switch ^= 1 << 4
print("{:08b}".format(switch))

00000001


In [39]:
# Toggle Switch 3 (XOR)
# You can pre-calculate (1 << 4) = 16 = 0b00010000 
switch ^= 16
print("{:08b}".format(switch))

00010001


You can manipulate bits using OR, AND, and XOR boolean operators. You can read bits using bit masks and bit shifts.